# Guia paso a paso para visualización de datos de la encuesta Ask A Manager Salary

Objetivo:
Extraer datos de diferentes fuentes (por ejemplo, bases de datos relacionales, APIs o archivos CSV), transformarlos para adecuarlos a nuestro modelo analítico, cargarlos en un data warehouse y, finalmente, crear dashboards interactivos en Looker Studio para la toma de decisiones.

Alcance:

Consolidar datos dispares en un único repositorio.
Garantizar la calidad y consistencia de los datos.
Proporcionar visualizaciones que permitan identificar tendencias y anomalías.

Nota: Para hacer la transformación correctamente, por favor ejecutar de forma secuencial todos los pasos

## 📥 Extracción de Datos
Cargamos el archivo CSV con los datos de la encuesta salarial. Primero, aseguramos que las librerías necesarias estén disponibles.


- Descarga del archivo:
Descarga el archivo disponible en el siguiente enlace:
Google Sheets - Ask_A_Manager
Asegúrate de exportarlo en formato CSV.
- Ubicación y nomenclatura:
Guarda el archivo descargado en la carpeta destinada al proyecto junto con el script en formato .ipynb. Es fundamental que el archivo se nombre exactamente 'Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1 (1).csv'.
- Inicio del proceso ETL:
Una vez guardado el archivo, abre el script para comenzar la transformación de forma automática.


### 📚 Librerías Necesarias
Importamos las librerías esenciales para el análisis.


In [13]:
import pandas as pd
import numpy as np

### 📂 Definición del Archivo de Datos
Especificamos el nombre del archivo CSV con los datos de la encuesta.


In [14]:
file_name = 'Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1 (1).csv'

### 📑 Carga de Datos
Leemos el archivo CSV en un DataFrame de Pandas.


In [15]:
# 📑 Cargar datos en un DataFrame
raw_data = pd.read_csv(file_name)
# Mostrar las primeras filas para verificar la carga correcta
raw_data.head()


,Timestamp,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White


# Verificación de los datos originales

In [16]:
print("Nombres de las columnas:")
print(raw_data.columns)

print("\nTipos de datos de cada columna:")
print(raw_data.dtypes)

Nombres de las columnas:
Index(['Timestamp', 'How old are you?', 'What industry do you work in?',
       'Job title',
       'If your job title needs additional context, please clarify here:',
       'What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)',
       'How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.',
       'Please indicate the currency',
       'If "Other," please indicate the currency here: ',
       'If your income needs additional context, please provide it here:',
       'What country do you work in?',
       'If you're in the U.S., what state do you work in?',
       'What city do you work in?',
       'How many years of professional work experience do yo

Se evidencia que las columnas estan en ingles y la mayoria son de tipo objeto excepto 'How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.' la cual es tipo float.

## 🔄 Transformación de Datos
Renombramos las columnas para hacerlas más manejables. También podríamos limpiar valores faltantes o corregir formatos si es necesario.


In [17]:
# 🔄 Renombrar columnas para facilitar el análisis
# Renombrar columnas
raw_data.rename(columns={'Timestamp': 'timestamp',
                         'How old are you?': 'age',
                         'What industry do you work in?': 'industry',
                         'Job title': 'job_title',
                         'If your job title needs additional context, please clarify here:': 'job_title_context',
                         "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)": 'annual_salary',
                         'How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.': 'additional_compensation',
                         'Please indicate the currency': 'currency',
                         'If "Other," please indicate the currency here: ': 'currency_other',
                         'If your income needs additional context, please provide it here:': 'income_context',
                         'What country do you work in?': 'country',
                         "If you're in the U.S., what state do you work in?": 'state_USA',
                         'What city do you work in?': 'city',
                         'How many years of professional work experience do you have overall?': 'years_experience',
                         'How many years of professional work experience do you have in your field?': 'years_experience_field',
                         'What is your highest level of education completed?': 'education',
                         'What is your gender?':'gender',
                         'What is your race? (Choose all that apply.)': 'race'}, inplace=True)

# Eliminar columnas no necesarias
columns_to_drop = ['job_title_context', 'income_context', 'state_USA','timestamp','city','job_title','race']
existing_columns_to_drop = [col for col in columns_to_drop if col in raw_data.columns]
raw_data.drop(columns=existing_columns_to_drop, inplace=True)
df = raw_data.copy()
df.dropna(subset=['age', 'industry', 'annual_salary', 'currency', 'country', 'years_experience', 'years_experience_field', 'education', 'gender'], inplace=True)


Mapeo de datos

In [18]:
# Creating a comprehensive mapping dictionary for standardizing the industries
industry_mapping_complete = {
    'Education (Higher Education)': 'Education',
    'Education (Primary/Secondary)': 'Education',
    'Education (early childhood)': 'Education',
    'University administration': 'Education',
    'Educational Technology - hybrid between book publishing and technology really': 'Education',
    'Academic research (Psychology)': 'Education',
    'Academic research (social science)': 'Education',
    'Academic Medicine ': 'Education',
    'STEM research': 'Education',
    'Research and development': 'Science & Research',
    'Scientific research': 'Science & Research',
    'Scientific R&D': 'Science & Research',
    'Science': 'Science & Research',
    'Science/Research': 'Science & Research',
    'Life Sciences': 'Science & Research',
    'Biotechnology': 'Science & Research',
    'Biotech': 'Science & Research',
    'Biotech/Pharma': 'Science & Research',
    'Biotech/pharmaceuticals': 'Science & Research',
    'Biomedical Research': 'Science & Research',
    'Biopharma': 'Science & Research',
    'Biotech Research': 'Science & Research',
    'Biotech / Research': 'Science & Research',
    'Biotechnology, Research and Development': 'Science & Research',
    'Computing or Tech': 'Technology',
    'Technology': 'Technology',
    'Tech': 'Technology',
    'Software as a Service SaaS': 'Technology',
    'Healthcare IT': 'Technology',
    'Software Development': 'Technology',
    'Software Development / IT': 'Technology',
    'IT': 'Technology',
    'Computing/Tech + Higher Ed + Nonprofit': 'Technology',
    'Accounting, Banking & Finance': 'Finance',
    'Finance': 'Finance',
    'Finance/Investment Management': 'Finance',
    'Insurance': 'Finance',
    'Health Insurance': 'Finance',
    'Nonprofits': 'Nonprofit',
    'Public Library': 'Nonprofit',
    'Library': 'Nonprofit',
    'Libraries': 'Nonprofit',
    'Museums': 'Nonprofit',
    'Archives': 'Nonprofit',
    'Publishing': 'Media & Publishing',
    'Publishing (academic)': 'Media & Publishing',
    'Publishing (book)': 'Media & Publishing',
    'Publishing: Science, Academic, Technical': 'Media & Publishing',
    'Media & Digital': 'Media & Publishing',
    'Digital': 'Media & Publishing',
    'Marketing, Advertising & PR': 'Marketing & PR',
    'Advertising': 'Marketing & PR',
    'Sales': 'Marketing & PR',
    'Retail': 'Retail & E-commerce',
    'E-commerce': 'Retail & E-commerce',
    'Online education': 'Education',
    'Real Estate': 'Real Estate',
    'Property or Construction': 'Real Estate',
    'Real estate': 'Real Estate',
    'Real estate software': 'Real Estate',
    'Real Estate/ Mortgage': 'Real Estate',
    'Government and Public Administration': 'Government',
    'Government': 'Government',
    'Government contractor': 'Government',
    'Government Relations': 'Government',
    'Government Relations/Lobbying': 'Government',
    'Law': 'Legal',
    'Legal services': 'Legal',
    'Law Enforcement & Security': 'Legal',
    'Law Library': 'Legal',
    'Public health': 'Healthcare',
    'Health care': 'Healthcare',
    'Medical Devices': 'Healthcare',
    'Medical communications': 'Healthcare',
    'Medical Research': 'Healthcare',
    'Medical Technology': 'Healthcare',
    'Veterinary medicine': 'Healthcare',
    'Pharmaceuticals': 'Pharmaceuticals',
    'Pharmaceutical': 'Pharmaceuticals',
    'Pharmaceutical R&D': 'Pharmaceuticals',
    'Pharmaceutical research': 'Pharmaceuticals',
    'Pharmaceutical Manufacturing': 'Pharmaceuticals',
    'Hospitality & Events': 'Hospitality',
    'Hospitality': 'Hospitality',
    'Food & Beverage': 'Food & Beverage',
    'Food': 'Food & Beverage',
    'Food Service': 'Food & Beverage',
    'Food manufacturing': 'Food & Beverage',
    'Consumer Packaged Goods': 'Consumer Goods',
    'Consumer Goods': 'Consumer Goods',
    'Manufacturing': 'Manufacturing',
    'Engineering or Manufacturing': 'Manufacturing',
    'Engineering': 'Manufacturing',
    'Manufacturing (medical devices)': 'Manufacturing',
    'Engineering - Mining': 'Mining',
    'Mining': 'Mining',
    'Aerospace': 'Aerospace & Defense',
    'Aerospace contracting': 'Aerospace & Defense',
    'Aerospace/aviation': 'Aerospace & Defense',
    'Aerospace & Defense': 'Aerospace & Defense',
    'Defense': 'Aerospace & Defense',
    'Defense contracting': 'Aerospace & Defense',
    'Military': 'Aerospace & Defense',
    'Oil and Gas': 'Energy',
    'Energy': 'Energy',
    'Renewable Energy': 'Energy',
    'Renewable energy': 'Energy',
    'Energy Supplier': 'Energy',
    'Telecommunications': 'Utilities & Telecommunications',
    'Utilities & Telecommunications': 'Utilities & Telecommunications',
    'Environmental Consulting': 'Environment',
    'Environmental sciences': 'Environment',
    'Environmental survey': 'Environment',
    'Environmental Science': 'Environment',
    'Environmental Planning': 'Environment',
    'Environmental compliance': 'Environment',
    'Environmental regulation': 'Environment',
    'Environmental/Cultural Resource Management': 'Environment',
    'Archaeology': 'Cultural Heritage',
    'Archaeologist': 'Cultural Heritage',
    'Cultural Resources Management/Major Univ.': 'Cultural Heritage',
    'Tourism': 'Tourism & Leisure',
    'Travel': 'Tourism & Leisure',
    'Leisure, Sport & Tourism': 'Tourism & Leisure',
    'Logistics': 'Transport & Logistics',
    'Transport or Logistics': 'Transport & Logistics',
    'Warehousing': 'Transport & Logistics',
    'Social Work': 'Social Services',
    'Human services': 'Social Services',
    'Childcare': 'Social Services',
    'Religious institution': 'Religious',
    'Religion': 'Religious',
    'Religious (synagogue)': 'Religious',
    'Music': 'Arts & Culture',
    'Art & Design': 'Arts & Culture',
    'Entertainment': 'Arts & Culture',
    'Performing Arts': 'Arts & Culture',
    'Theatre': 'Arts & Culture',
}

mapping_country = {
    # Estados Unidos
    'United States': 'United States',
    'US': 'United States',
    'USA': 'United States',
    'usa': 'United States',
    'us': 'United States',
    'Usa': 'United States',
    'United States ': 'United States',
    'United states': 'United States',
    'united states': 'United States',
    'USA ': 'United States',
    'United States of America': 'United States',
    'United States of America ': 'United States',
    'united states of america': 'United States',
    'The United States': 'United States',
    'United State': 'United States',
    'U.S.A': 'United States',
    'United Stated': 'United States',
    'Unites States ': 'United States',
    'United Sates': 'United States',
    'United Statea': 'United States',
    'United Statees': 'United States',
    'United States of Americas': 'United States',
    'United States of American ': 'United States',
    'Uniited States': 'United States',
    'United Statesp': 'United States',
    'United Stattes': 'United States',
    'United Statws': 'United States',
    'United Statss': 'United States',
    'United Statues': 'United States',
    'Unted States': 'United States',
    'Uniyed states': 'United States',
    'Uniyes States': 'United States',
    'Unitied States': 'United States',
    'United  States': 'United States',
    'US of A': 'United States',
    'USA tomorrow ': 'United States',
    'UnitedStates': 'United States',
    'USA-- Virgin Islands': 'United States Virgin Islands',
    'U.S.': 'United States',
    'U.S. ': 'United States',
    'U.S': 'United States',
    'U. S.': 'United States',
    'U. S. ': 'United States',
    'u.s.': 'United States',
    'u.s.': 'United States',
    'U. S': 'United States',
    'U.SA': 'United States',
    'U.s.a.': 'United States',
    'USAB': 'United States',
    'U.S.A.': 'United States',
    'U.S.A ': 'United States',
    'USaa': 'United States',
    'uSA': 'United States',
    'Us ': 'United States',
    'US ': 'United States',
    'United statew': 'United States',
    'United Statss': 'United States',
    'United Sttes': 'United States',
    'United Statues': 'United States',
    'United states of America ': 'United States',
    'United States Of America': 'United States',
    'United states of America': 'United States',
    'United States is America': 'United States',
    'United  States': 'United States',
    'United Stareds': 'United States',  # si apareciera, se mapea igual
    'Uniter Statez': 'United States',
    'United Stateds': 'United States',
    'Untied States': 'United States',
    'United Stares ': 'United States',
    'United Statss': 'United States',
    
    # Reino Unido (se agrupan variantes de United Kingdom, England, Scotland, Northern Ireland, Wales, etc.)
    'United Kingdom': 'United Kingdom',
    'United Kingdom ': 'United Kingdom',
    'UK': 'United Kingdom',
    'UK ': 'United Kingdom',
    'Uk': 'United Kingdom',
    'U.K.': 'United Kingdom',
    'U.K. ': 'United Kingdom',
    'U.K': 'United Kingdom',
    'England': 'United Kingdom',
    'England ': 'United Kingdom',
    'England, UK.': 'United Kingdom',
    'England, United Kingdom': 'United Kingdom',
    'England/UK': 'United Kingdom',
    'United Kingdom (England)': 'United Kingdom',
    'United Kindom': 'United Kingdom',
    'United kingdom': 'United Kingdom',
    'United kingdom ': 'United Kingdom',
    'England, Gb': 'United Kingdom',
    'UK (England)': 'United Kingdom',
    'England, UK': 'United Kingdom',
    'United Kingdom.': 'United Kingdom',
    'Great Britain ': 'United Kingdom',
    'Great Britain': 'United Kingdom',
    'Britain ': 'United Kingdom',
    'England, United Kingdom ': 'United Kingdom',
    'United Kingdomk': 'United Kingdom',
    'UK, remote': 'United Kingdom',
    'England/UK': 'United Kingdom',
    'Scotland ': 'United Kingdom',
    'Scotland': 'United Kingdom',
    'Scotland, UK': 'United Kingdom',
    'Northern Ireland': 'United Kingdom',
    'Northern Ireland ': 'United Kingdom',
    'UK (Northern Ireland)': 'United Kingdom',
    'Wales (United Kingdom)': 'United Kingdom',
    'Wales (UK)': 'United Kingdom',
    'Wales': 'United Kingdom',
    'Wales, UK': 'United Kingdom',
    'UK, but for globally fully remote company': 'United Kingdom',
    'England, Gb': 'United Kingdom',
    'U.K. (northern England)': 'United Kingdom',
    'London': 'United Kingdom',
    
    # Canadá
    'Canada': 'Canada',
    'canada': 'Canada',
    'Canada ': 'Canada',
    'Canada, Ottawa, ontario': 'Canada',
    'I am located in Canada but I work for a company in the US': 'Canada',
    'CANADA ': 'Canada',
    'CANADA': 'Canada',
    'Csnada': 'Canada',
    'Canad': 'Canada',
    'Canda': 'Canada',
    'Canadá': 'Canada',
    'Canada and USA': 'Canada and United States',
    
    # Australia
    'Australia ': 'Australia',
    'Australia': 'Australia',
    'australia': 'Australia',
    'Australi': 'Australia',
    'Australian ': 'Australia',
    
    # Alemania
    'Germany': 'Germany',
    'germany': 'Germany',
    'Germany ': 'Germany',
    
    # Irlanda
    'Ireland': 'Ireland',
    'Ireland ': 'Ireland',
    'ireland': 'Ireland',
    
    # Finlandia
    'finland': 'Finland',
    'Finland': 'Finland',
    
    # India
    'India': 'India',
    'INDIA': 'India',
    
    # Argentina
    'Argentina': 'Argentina',
    "I work for an US based company but I'm from Argentina.": 'Argentina',
    'ARGENTINA BUT MY ORG IS IN THAILAND': 'Argentina',
    
    # Francia
    'France': 'France',
    'FRANCE': 'France',
    'France ': 'France',
    'france': 'France',
    
    # Países Bajos
    'The Netherlands': 'Netherlands',
    'Netherlands': 'Netherlands',
    'netherlands': 'Netherlands',
    'Netherlands ': 'Netherlands',
    'The Netherlands ': 'Netherlands',
    'the Netherlands': 'Netherlands',
    'The netherlands': 'Netherlands',
    
    # Dinamarca
    'Denmark': 'Denmark',
    'Denmark ': 'Denmark',
    'denmark': 'Denmark',
    'Danmark': 'Denmark',
    
    # Suecia
    'Sweden': 'Sweden',
    'Sweden ': 'Sweden',
    
    # Hong Kong
    'Hong Kong': 'Hong Kong',
    'Hong Kong ': 'Hong Kong',
    
    # Kuwait
    'Kuwait': 'Kuwait',
    
    # Noruega
    'Norway': 'Norway',
    'Norway ': 'Norway',
    
    # Sri Lanka
    'Sri lanka': 'Sri Lanka',
    'Sri Lanka': 'Sri Lanka',
    
    # Otros – respuestas ambiguas o sin país claro
    'Contracts': 'Other',
    "We don't get raises, we get quarterly bonuses, but they periodically asses income in the area you work, so I got a raise because a 3rd party assessment showed I was paid too little for the area we were located": 'Other',
    'Global': 'Other',
    'Currently finance': 'Other',
    '$2,175.84/year is deducted for benefits': 'Other',
    "I was brought in on this salary to help with the EHR and very quickly was promoted to current position but compensation was not altered. ": 'Other',
    'UXZ': 'Other',
    'ss': 'Other',
    'dbfemf': 'Other',
    'LOUTRELAND': 'Other',
    
    # Otros países y respuestas específicas
    'Malaysia': 'Malaysia',
    'Mexico ': 'Mexico',
    'Mexico': 'Mexico',
    'México': 'Mexico',
    'South Africa ': 'South Africa',
    'South Africa': 'South Africa',
    'Belgium': 'Belgium',
    'Belgium ': 'Belgium',
    'Bermuda': 'Bermuda',
    'Greece': 'Greece',
    'Austria': 'Austria',
    'Brazil': 'Brazil',
    'Brazil ': 'Brazil',
    'Trinidad and Tobago': 'Trinidad and Tobago',
    'Cayman Islands': 'Cayman Islands',
    'I work for a UAE-based organization, though I am personally in the US.': 'United States',
    'Uniyed states': 'United States',
    'Uniyes States': 'United States',
    'U.A.': 'United Arab Emirates',
    'United Arab Emirates ': 'United Arab Emirates',
    'Romania': 'Romania',
    'Serbia': 'Serbia',
    'Philippines': 'Philippines',
    'Russia ': 'Russia',
    'Poland': 'Poland',
    'czech republic': 'Czech Republic',
    'Czech republic': 'Czech Republic',
    'Czechia': 'Czech Republic',
    'Latvia': 'Latvia',
    'Puerto Rico': 'Puerto Rico',
    'Rwanda': 'Rwanda',
    'U.SA': 'United States',
    'United Kindom': 'United Kingdom',
    'United Status': 'United States',
    'New Zealand': 'New Zealand',
    'New Zealand ': 'New Zealand',
    ' U.S.': 'United States',
    'Serbia': 'Serbia',
    'Poland': 'Poland',
    'Turkey': 'Turkey',
    'Puerto Rico ': 'Puerto Rico',
    'Catalonia': 'Spain',
    'uk': 'United Kingdom',
    'Italy (South)': 'Italy',
    'Jersey, Channel islands': 'Jersey',
    'Uk ': 'United Kingdom',
    'China': 'China',
    'Virginia': 'United States',
    'Afghanistan': 'Afghanistan',
    'Israel': 'Israel',
    'U.s.': 'United States',
    'U.s.a.': 'United States',
    'USS': 'United States',
    'Denmark ': 'Denmark',
    'Uniteed States': 'United States',
    'New Zealand Aotearoa': 'New Zealand',
    'Hartford': 'United States',
    'Japan, US Gov position': 'Japan',
    'New zealand': 'New Zealand',
    'Mainland China': 'China',
    'I.S.': 'Other',
    'UK (Northern Ireland)': 'United Kingdom',
    'UK for U.S. company': 'United Kingdom',
    'Japan': 'Japan',
    ' US': 'United States',
    'Unites states ': 'United States',
    'NZ': 'New Zealand',
    'Taiwan': 'Taiwan',
    'Canad': 'Canada',
    'Unite States': 'United States',
    'The US': 'United States',
    'The Netherlands ': 'Netherlands',
    'Cambodia': 'Cambodia',
    'Vietnam': 'Vietnam',
    'Remote': 'Remote',
    'Singapore': 'Singapore',
    'South Korea': 'South Korea',
    'Czech Republic ': 'Czech Republic',
    'Thailand': 'Thailand',
    'Bangladesh': 'Bangladesh',
    'Lithuania': 'Lithuania',
    'Eritrea': 'Eritrea',
    'Indonesia': 'Indonesia',
    'Singapore ': 'Singapore',
    'For the United States government, but posted overseas': 'Other',
    'IS': 'Other',
    'Switzerland ': 'Switzerland',
    'Italy': 'Italy',
    ' New Zealand': 'New Zealand',
    'Cuba': 'Cuba',
    'Slovenia': 'Slovenia',
    'Australi': 'Australia',
    "Cote d'Ivoire": "Côte d'Ivoire",
    'From Romania, but for an US based company': 'Romania',
    'Somalia': 'Somalia',
    'Wales (United Kingdom)': 'United Kingdom',
    'England, Gb': 'United Kingdom',
    'Czech Republic': 'Czech Republic',
    'UnitedStates': 'United States',
    'Danmark': 'Denmark',
    'U.K. (northern England)': 'United Kingdom',
    'NL': 'Netherlands',
    'the Netherlands': 'Netherlands',
    'Nederland': 'Netherlands',
    'Slovakia': 'Slovakia',
    'Portugal ': 'Portugal',
    'Sierra Leone': 'Sierra Leone',
    'Englang': 'United Kingdom',
    'UAE': 'United Arab Emirates',
    'International ': 'Other',
    'The Bahamas ': 'The Bahamas',
    'I earn commission on sales. If I meet quota, I\'m guaranteed another 16k min. Last year i earned an additional 27k. It\'s not uncommon for people in my space to earn 100k+ after commission. ': 'Other',
    'Costa Rica': 'Costa Rica',
    ' United States': 'United States',
    'United Statues': 'United States',
    'Untied States': 'United States',
    'Chile': 'Chile',
    'UK (England)': 'United Kingdom',
    'UK, remote': 'United Kingdom',
    'Scotland, UK': 'United Kingdom',
    'Unitied States': 'United States',
    'Qatar': 'Qatar',
    'Remote (philippines)': 'Philippines',
    'Brazil ': 'Brazil',
    'Unites kingdom ': 'United Kingdom',
    'South africa': 'South Africa',
    'united stated': 'United States',
    'United States Of America': 'United States',
    'Nigeria': 'Nigeria',
    'Panamá': 'Panama',
    'Northern Ireland ': 'United Kingdom',
    'england': 'United Kingdom',
    'SWITZERLAND': 'Switzerland',
    'Austria, but I work remotely for a Dutch/British company': 'Austria',
    "I work for an US based company but I'm from Argentina.": 'Argentina',
    "I was brought in on this salary to help with the EHR and very quickly was promoted to current position but compensation was not altered. ": 'Other',
    'Uniter Statez': 'United States',
    'U. S ': 'United States',
    'Congo': 'Congo',
    'Uruguay': 'Uruguay',
    'Pakistan': 'Pakistan',
    'Brasil': 'Brazil',
    'n/a (remote from wherever I want)': 'Other',
    'singapore': 'Singapore',
    'US govt employee overseas, country withheld': 'Other',
    'usa ': 'United States',
    'Malaysia ': 'Malaysia',
    'Uganda': 'Uganda',
    'Malta': 'Malta',
    'Saudi Arabia': 'Saudi Arabia',
    'Bulgaria': 'Bulgaria',
    'Estonia': 'Estonia',
    'Morocco': 'Morocco',
    'new zealand': 'New Zealand',
    'Africa': 'Other',
    'Ecuador': 'Ecuador',
    'Zimbabwe': 'Zimbabwe',
    'Ghana': 'Ghana',
    'San Francisco': 'United States',
    'Japan ': 'Japan',
    'Usat': 'United States',
    '🇺🇸 ': 'United States',
    'Luxemburg': 'Luxembourg',
    'Unitef Stated': 'United States',
    'UA': 'Ukraine',
    'Wales, UK': 'United Kingdom',
    'Croatia ': 'Croatia',
    'England, United Kingdom ': 'United Kingdom',
    'United STates': 'United States',
    'USaa': 'United States',
    'South Korea ': 'South Korea',
    'The netherlands': 'Netherlands',
    'Ukraine ': 'Ukraine',
    'spain': 'Spain',
    'america': 'United States',
    'switzerland': 'Switzerland',
    'United States- Puerto Rico': 'United States',
    'From New Zealand but on projects across APAC': 'New Zealand',
    'Y': 'Other',
    'United y': 'Other',
    'Wales (UK)': 'United Kingdom',
    'Isle of Man': 'Isle of Man',
    'Northern Ireland, United Kingdom': 'United Kingdom',
    'europe': 'Other',
    'California ': 'United States',
    'Australian ': 'Australia',
    'UK, but for globally fully remote company': 'United Kingdom',
    'Jamaica': 'Jamaica',
    'uS': 'United States',
    'USD': 'Other',
    'the netherlands': 'Netherlands',
    "USA, but for foreign gov't": 'United States',
    'japan': 'Japan',
    'Kenya': 'Kenya',
    'Jordan': 'Jordan',
    'United Statss': 'United States',
    'United states of america': 'United States',
    'UsA': 'United States',
    'I work for a UAE-based organization, though I am personally in the US.': 'United States',
    'france': 'France',
    'Aotearoa New Zealand': 'New Zealand',
    'na': 'Other',
    'Policy': 'Other',
    'Cyprus': 'Cyprus',
    'hong konh': 'Hong Kong',
    'Liechtenstein': 'Liechtenstein',
    'Company in Germany. I work from Pakistan.': 'Pakistan',
    'croatia': 'Croatia',
    'ENGLAND': 'United Kingdom',
    'United States of American': 'United States',
    'INDIA': 'India',
    'NIGERIA': 'Nigeria',
    'Poland ': 'Poland',
    'pakistan': 'Pakistan',
    'Nigeria ': 'Nigeria',
    'ss': 'Other',
    'dbfemf': 'Other',
    'LOUTRELAND': 'Other',
    'philippines': 'Philippines',
    'Myanmar': 'Myanmar',
    'Burma': 'Myanmar',
    'Tanzania': 'Tanzania',
    'Česká republika': 'Czech Republic',
    'Italia': 'Italy'
}


# Diccionario de mapeo para la moneda
currency_mapping = {
    # Valores de tipo moneda o código:
    'INR': 'INR',
    'Peso Argentino': 'ARS',
    '$76,302.34': 'USD',
    'MYR': 'MYR',
    'CHF': 'CHF',
    'KWD': 'KWD',
    'NOK': 'NOK',
    'USD': 'USD',
    'BR$': 'BRL',
    'SEK': 'SEK',
    'canadian': 'CAD',
    'Dkk': 'DKK',
    'EUR': 'EUR',
    'COP': 'COP',
    'TTD': 'TTD',
    'Indian rupees': 'INR',
    'BRL (R$)': 'BRL',
    'Mexican pesos': 'MXN',
    'CZK': 'CZK',
    'GBP': 'GBP',
    'DKK': 'DKK',
    'ZAR': 'ZAR',
    'American Dollars': 'USD',
    'Php': 'PHP',
    'PLN (Polish zloty)': 'PLN',
    'czech crowns': 'CZK',
    'TRY': 'TRY',
    'Norwegian kroner (NOK)': 'NOK',
    'CNY': 'CNY',
    'ILS/NIS': 'ILS',
    'US Dollar': 'USD',
    'Canadian ': 'CAD',
    'AUD': 'AUD',
    'BRL': 'BRL',
    'NIS (new Israeli shekel)': 'ILS',
    'RMB (chinese yuan)': 'CNY',
    'Taiwanese dollars': 'TWD',
    'NZD': 'NZD',
    'Philippine Peso': 'PHP',
    'SGD': 'SGD',
    'KRW (Korean Won)': 'KRW',
    'Czk': 'CZK',
    'THB': 'THB',
    'IDR ': 'IDR',
    'Sgd': 'SGD',
    'Nok': 'NOK',
    'ILS (Shekel)': 'ILS',
    'DKK ': 'DKK',
    'China RMB': 'CNY',
    'AUD Australian ': 'AUD',
    'LKR': 'LKR',
    'Polish Złoty': 'PLN',
    'Philippine peso (PHP)': 'PHP',
    'Australian Dollars ': 'AUD',
    'PHP': 'PHP',
    'ARS': 'ARS',
    'Argentinian peso (ARS)': 'ARS',
    'Israeli Shekels': 'ILS',
    'ILS': 'ILS',
    'MXN': 'MXN',
    'PhP (Philippine Peso)': 'PHP',
    'Converted mine into USD for your easyness': 'USD',
    'PLN': 'PLN',
    'KRW': 'KRW',
    'SAR': 'SAR',
    'RM': 'MYR',
    'IDR': 'IDR',
    'Argentine Peso': 'ARS',
    'Philippine Pesos': 'PHP',
    'Ils': 'ILS',
    'Rs': 'INR',
    'INR (Indian Rupee)': 'INR',
    'Danish Kroner': 'DKK',
    'CAD': 'CAD',
    'Korean Won ': 'KRW',
    'dkk': 'DKK',
    'Euro': 'EUR',
    'SGD ': 'SGD',
    'NTD': 'TWD',
    'Mexican Pesos': 'MXN',
    'THAI  BAHT': 'THB',
    'Thai Baht ': 'THB',
    'Canadian': 'CAD',
    'croatian kuna': 'HRK',
    'PLN (Zwoty)': 'PLN',
    'Rupees': 'INR',
    'Singapore Dollara': 'SGD',
    'NGN': 'NGN',
    'pkr': 'PKR',
    'HKD': 'HKD',
    'Rupees ': 'INR',
    'TZS': 'TZS',
    'Pesos Colombianos': 'COP',

    # Valores que no representan una moneda (comentarios, estructuras de compensación, números puros, etc.)
    'My bonus is based on performance up to 10% of salary': 'Other',
    'I work for an online state university, managing admissions data. Not direct tech support. ': 'Other',
    '0': 'Other',
    'Na ': 'Other',
    'Base plus Commission ': 'Other',
    'RSU / equity': 'Other',
    'Additonal = Bonus plus stock': 'Other',
    'Overtime (about 5 hours a week) and bonus': 'Other',
    'Stock ': 'Other',
    '55,000': 'Other',
    'AUD & NZD are not the same currency...': 'Other',
    '-': 'Other',
    "AUD and NZD aren't the same currency, and have absolutely nothing to do with each other :(": 'Other',
    '6000 in stock grants annually': 'Other',
    'Many non-salary benefits - travel, free healthcare for self, very low for family, non-taxable housing allowance ': 'Other',
    'Equity': 'Other',
    'It’s marketed as £22000 but we get paid pro-rats, so no pay for the school holidays.': 'Other',
    'additional compensation is for overtime (i am paid hourly) so it varies. i have included an estimate': 'Other',
    'Option to get 2x or 1.5x if taking on a weekend day in the summer': 'Other',
    '47000': 'Other',
    'na': 'Other',
    'N/a': 'Other',
    'up to 12% annual bonus': 'Other',
    '5': 'Other',
    '1': 'Other',
    'ekignkfb': 'Other',
    'other': 'Other'
}


# Apply the mapping to the full dataset
df['industry'] = df['industry'].map(industry_mapping_complete).fillna('Other')
df['country'] = df['country'].map(mapping_country).fillna('Other')
df['currency_other'] = df['currency_other'].map(currency_mapping).fillna('Other')

Conversion de datos a numericos

In [19]:
df['annual_salary'] = df['annual_salary'].str.replace(',','').astype(float)
df['additional_compensation'] = df['additional_compensation'].astype(str)
df['additional_compensation'] = df['additional_compensation'].str.replace(',','').astype(float)
df['additional_compensation'] = df['additional_compensation'].fillna(0)
df['total_salary'] = df['annual_salary'] + df['additional_compensation']

# Adicion de valores cuando currency es USD y tiene menos de 3 digitos
mask = (df['total_salary'] >= 1) & (df['total_salary'] < 1000)
df.loc[mask, 'total_salary'] *= 1000
df.drop(columns=['annual_salary', 'additional_compensation'], inplace=True)

In [20]:
# Estandarizacion de currency
df.loc[df['currency'] == 'Other', 'currency'] = df.loc[df['currency'] == 'Other', 'currency_other']
df.drop(columns=['currency_other'], inplace=True)
df = df[df['currency'] != 'Other']

# Diccionario de tasas de conversion
conversion_rates = {
    'USD': 4137,
    'GBP': 5152,
    'CAD': 2885,
    'EUR': 4293,
    'AUD/NZD': 2800, 
    'INR': 47,
    'ARS': 3,
    'CHF': 4557,
    'MYR': 931,
    'ZAR': 221,
    'SEK': 378,
    'HKD': 525,
    'NOK': 369,
    'BRL': 717,
    'DKK': 572,
    'TTD': 611,
    'MXN': 201,
    'CZK': 171,
    'PHP': 71,
    'PLN': 1024,
    'TRY': 300,
    'CNY': 567,
    'ILS': 1163,
    'AUD': 2600,
    'JPY': 27,
    'TWD': 125,
    'SGD': 3062,
    'KRW': 2.8,
    'THB': 123,
    'IDR': 0.2,
    'NZD': 2325,
    'LKR': 13,
    'SAR': 1100,
    'HRK': 574,
    'NGN': 2.8,
    'TZS': 1.6,
    'COP': 1 
}

df['salary_COP'] = df.apply(
    lambda row: row['total_salary'] * conversion_rates.get(row['currency'], 1),
    axis=1
)


# Datos finales:

In [21]:
print("Nombres de las columnas:")
print(df.columns)

print("\nTipos de datos de cada columna:")
print(df.dtypes)

Nombres de las columnas:
Index(['age', 'industry', 'currency', 'country', 'years_experience',
       'years_experience_field', 'education', 'gender', 'total_salary',
       'salary_COP'],
      dtype='object')

Tipos de datos de cada columna:
age                        object
industry                   object
currency                   object
country                    object
years_experience           object
years_experience_field     object
education                  object
gender                     object
total_salary              float64
salary_COP                float64
dtype: object


Se convierte las variables, manteniendo el idioma inglés pero abreviando sus descripciones. Además, se crean dos nuevas columnas, 'total_salary' y 'salary_COP', definidas como tipo float, en contraste con las demás que se mantienen como tipo objeto.

# Load

Exportar los datos a un archivo CSV

In [22]:
df.to_csv('cleaned_data.csv', index=False)

Vista general de la estructura de los datos:

In [24]:
df.head()

,age,industry,currency,country,years_experience,years_experience_field,education,gender,total_salary,salary_COP
0,25-34,Education,USD,United States,5-7 years,5-7 years,Master's degree,Woman,55000.0,227535000.0
1,25-34,Technology,GBP,United Kingdom,8 - 10 years,5-7 years,College degree,Non-binary,58600.0,301907200.0
2,25-34,Finance,USD,United States,2 - 4 years,2 - 4 years,College degree,Woman,34000.0,140658000.0
3,25-34,Nonprofit,USD,United States,8 - 10 years,5-7 years,College degree,Woman,65000.0,268905000.0
4,25-34,Finance,USD,United States,8 - 10 years,5-7 years,College degree,Woman,67000.0,277179000.0
